In [5]:
import sqlite3

conn = sqlite3.connect("cafe.db")
cursor = conn.cursor()

# Create a table for menu items
cursor.execute('''CREATE TABLE IF NOT EXISTS menu (
                item_id INTEGER PRIMARY KEY,
                item_name TEXT,
                item_price REAL)''')

# Create a table for orders
cursor.execute('''CREATE TABLE IF NOT EXISTS orders (
                order_id INTEGER PRIMARY KEY,
                item_id INTEGER,
                quantity INTEGER,
                total_price REAL)''')

conn.commit()
conn.close()

In [6]:
import tkinter as tk
import sqlite3

# Function to add an item to the order
def add_item():
    selected_item = menu_listbox.get(tk.ACTIVE)
    quantity = quantity_entry.get()

    if selected_item and quantity.isdigit():
        item_id = menu_items[selected_item]
        item_price = menu_prices[selected_item]
        quantity = int(quantity)
        total_price = item_price * quantity

        # Add the order to the database
        conn = sqlite3.connect("cafe.db")
        cursor = conn.cursor()
        cursor.execute("INSERT INTO orders (item_id, quantity, total_price) VALUES (?, ?, ?)",
                       (item_id, quantity, total_price))
        conn.commit()
        conn.close()

        # Update the order list
        order_listbox.insert(tk.END, f"{selected_item} x{quantity}")

        # Update the total bill
        total_bill.set(f"Total Bill: ${calculate_total_bill()}")

# Function to calculate the total bill
def calculate_total_bill():
    conn = sqlite3.connect("cafe.db")
    cursor = conn.cursor()
    cursor.execute("SELECT SUM(total_price) FROM orders")
    total = cursor.fetchone()[0]
    conn.close()
    return total if total else 0.0

# Create the main window
window = tk.Tk()
window.title("Cafe Management System")

# Create a list of menu items and their prices
menu_items = {}
menu_prices = {"Coffee": 2.5, "Tea": 2.0, "Cake": 3.0, "Sandwich": 5.0}

# Create a menu listbox
menu_listbox = tk.Listbox(window)
for item in menu_prices.keys():
    menu_items[item] = len(menu_items) + 1
    menu_listbox.insert(tk.END, item)
menu_listbox.pack()

# Create an entry for quantity
quantity_label = tk.Label(window, text="Quantity:")
quantity_label.pack()
quantity_entry = tk.Entry(window)
quantity_entry.pack()

# Create a button to add items to the order
add_button = tk.Button(window, text="Add to Order", command=add_item)
add_button.pack()

# Create a listbox to display the order
order_listbox = tk.Listbox(window)
order_listbox.pack()

# Create a label to display the total bill
total_bill = tk.StringVar()
total_bill.set(f"Total Bill: ${calculate_total_bill()}")
total_bill_label = tk.Label(window, textvariable=total_bill)
total_bill_label.pack()

# Start the Tkinter main loop
window.mainloop()
